In [1]:
import numpy as np
import pickle
data_path="../Results/cost_matrices_30.npz"
assignments_path="../Results/assignments.pkl"

data=np.load(data_path)
one_matrix=data["one_matrix"][0]
two_matrix=data["two_matrix"][0]
three_matrix=data["three_matrix"][0]
expectation_matrix=data["expectation_matrix"]
routes=data["routes"][0]
assignments=pickle.load(open(assignments_path,"rb"))

In [ ]:
def get_route_cost(route, cost_matrix_one, cost_matrix_two, cost_matrix_three, order_number, two_combinations_map, three_combinations_map, routes):
    """
    Query the cost of a single route.

    Args:
        route (list): Single route information, format is [[order batch], driver ID]. E.g., [[18, 25], 7]
        cost_matrix (np.array): Matrix used to query cost (e.g., one_matrix[0]).
        order_number (int): Total number of orders in the scenario (e.g., 30).
        two_combinations_map (list): Mapping list from 2-order combinations to indices.
        three_combinations_map (list): Mapping list from 3-order combinations to indices.

    Returns:
        float: The cost of the route.
    """
    # Separate batch and driver from route information
    batch = route[0]
    driver = route[1]

    # Sort the batch and convert to tuple to match the format in combination files
    sorted_batch = tuple(sorted(batch))

    batch_index = -1

    if len(sorted_batch) == 1:
        # Single order batch, index is the order ID itself
        batch_index = sorted_batch[0]
        print(f"Driver {driver} executing order {sorted_batch} has path cost: {cost_matrix_one[driver][batch_index]:.4f}")
        return cost_matrix_one[driver][batch_index], 0
    elif len(sorted_batch) == 2:
        # Two-order batch, find its index in the mapping list, and add total orders as offset
        try:
            batch_index = two_combinations_map.index(sorted_batch)
            current_route = routes[driver][order_number + batch_index]
            if current_route == 0:
                route_cost = np.sum(cost_matrix_two[driver, sorted_batch[0], sorted_batch[1]])
                print(f"Driver {driver} executing order {sorted_batch} has path cost: {route_cost:.4f}")
                return route_cost, current_route
            else:
                route_cost = np.sum(cost_matrix_two[driver, sorted_batch[1], sorted_batch[0]])
                print(f"Driver {driver} executing order {sorted_batch} has expected path cost: {route_cost:.4f}")
                return route_cost, current_route
        except ValueError:
            raise ValueError(f"Could not find batch in 2-order combinations: {sorted_batch}")
    elif len(sorted_batch) == 3:
        # Three-order batch, add total orders and total 2-order combinations as offset
        order_permutations = [[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]]
        try:
            batch_index = three_combinations_map.index(sorted_batch)
            current_route = routes[driver, order_number + len(two_combinations_map) + batch_index]
            index1, index2, index3 = order_permutations[current_route]
            route_cost = np.sum(cost_matrix_three[driver, sorted_batch[index1], sorted_batch[index2], sorted_batch[index3]])
            print(f"Driver {driver} executing order {sorted_batch} has path cost: {route_cost:.4f}")
            return route_cost, current_route
        except ValueError:
            raise ValueError(f"Could not find batch in 3-order combinations: {sorted_batch}")

    if batch_index == -1:
        raise ValueError(f"Batch {sorted_batch} format is incorrect.")


def get_total_objective(
    assignment, cost_matrix_one, cost_matrix_two, cost_matrix_three, order_number, two_combinations_map, three_combinations_map, routes
):
    """
    Calculate the total objective value of the entire assignment plan (sum of costs of all routes).

    Args:
        assignment (list): Complete assignment plan, a list of multiple routes.
        cost_matrix (np.array): Matrix used to query costs.
        order_number (int): Total number of orders in the scenario.
        two_combinations_map (list): Mapping list from 2-order combinations to indices.
        three_combinations_map (list): Mapping list from 3-order combinations to indices.

    Returns:
        float: Total cost of the entire plan.
    """
    total_cost = 0
    # Iterate through each route in the plan
    for route in assignment:
        # Call the function above to calculate cost of single route and accumulate
        cost, current_route = get_route_cost(
            route, cost_matrix_one, cost_matrix_two, cost_matrix_three, order_number, two_combinations_map, three_combinations_map, routes
        )
        total_cost += cost
    return total_cost

In [ ]:
# --- 1. Define your scenario parameters ---
ORDER_NUMBER = 30
# Assuming your .npz and .pkl files only contain data for this one scenario, so matrix index is 0
SCENARIO_INDEX = 0

# --- 2. Load extra 'combinations' mapping files ---
# This is necessary to map combinations like (Order1, Order2) to cost matrix indices
combinations_path = "../Resources/combinations/"
with open(f"{combinations_path}{ORDER_NUMBER}_2.pkl", "rb") as f:
    two_combinations_map = pickle.load(f)
with open(f"{combinations_path}{ORDER_NUMBER}_3.pkl", "rb") as f:
    three_combinations_map = pickle.load(f)

# assignments is a list, each element corresponds to a solution for a scenario, so we also choose the first one [0]
# Assuming you want to analyze the results of the 'routing' model
assignment_to_check = assignments[SCENARIO_INDEX]["routing"]

# --- 4. Call function and print results ---

# Calculate and print total objective value
total_obj_value = get_total_objective(
    assignment=assignment_to_check,
    cost_matrix_one=one_matrix,
    cost_matrix_two=two_matrix,
    cost_matrix_three=three_matrix,
    order_number=ORDER_NUMBER,
    two_combinations_map=two_combinations_map,
    three_combinations_map=three_combinations_map,
    routes=routes,
)

print(f"Total Objective Value of Assignment Plan: {total_obj_value:.4f}")
print("-" * 30)

# Get and print cost of a specific route
# We take the first route in the assignment plan as an example
if assignment_to_check:
    sample_route = assignment_to_check[5]

    sample_route_cost, current_route = get_route_cost(
        route=sample_route,
        cost_matrix_one=one_matrix,
        cost_matrix_two=two_matrix,
        cost_matrix_three=three_matrix,
        order_number=ORDER_NUMBER,
        two_combinations_map=two_combinations_map,
        three_combinations_map=three_combinations_map,
        routes=routes,
    )

    print("Single route cost query example:")
    print(f"  - Route queried (Order batch, Driver): {sample_route}")
    print(f"  - Cost of this route is: {sample_route_cost:.4f}")
    print(f"  - Current status of this route is: {current_route}")
else:
    print("Assignment plan is empty.")

司机 0 执行订单 (0, 11, 25) 的路径成本为: 2.9616
司机 1 执行订单 (7, 13, 23) 的路径成本为: 3.4268
司机 2 执行订单 (1, 9, 10) 的路径成本为: 5.6166
司机 3 执行订单 (8, 20, 26) 的路径成本为: 0.3378
司机 4 执行订单 (12, 18, 24) 的路径成本为: 12.1219
司机 5 执行订单 (4, 14, 15) 的路径成本为: 0.4556
司机 6 执行订单 (3, 5, 27) 的路径成本为: 3.6032
司机 7 执行订单 (6, 17, 19) 的路径成本为: 1.0978
司机 8 执行订单 (21, 22, 29) 的路径成本为: 32.1734
司机 9 执行订单 (2, 16, 28) 的路径成本为: 0.2820
分配方案的总目标值 (Total Objective): 62.0768
------------------------------
司机 5 执行订单 (4, 14, 15) 的路径成本为: 0.4556
单条路径成本查询示例:
  - 查询的路径 (订单批次, 司机): [array([ 4, 14, 15]), 5]
  - 该路径的成本是: 0.4556
  - 该路径的当前状态是: 1


In [ ]:
print("--- Verify relationship: one_matrix[d, i] == two_matrix[d, i, j, 0] ---")
print("-" * 70)

# Get dimension info from loaded one_matrix
num_drivers, num_orders = one_matrix.shape

# Define some random test cases (driver index, first order index, second order index)
# Ensure these indices are within your data range (e.g., driver < 15, order < 30)
test_cases = [
    (0, 5, 10),  # Test driver 0, first order 5, second order 10
    (7, 12, 3),  # Test driver 7, first order 12, second order 3
    (14, 29, 0)  # Test boundary case (last driver and last order)
]

all_passed = True
for d, i, j in test_cases:
    print(f"[*] Checking: Driver(d)={d}, First Order(i)={i}, Second Order(j)={j}")
    
    # Get delay value for single leg trip from one_matrix
    value_from_one_matrix = one_matrix[d, i]
    
    # Get delay value for *first leg* of two-leg trip from two_matrix
    value_from_two_matrix = two_matrix[d, i, j, 0]
    
    print(f"    - Value of one_matrix[{d}, {i}]: {value_from_one_matrix:.6f}")
    print(f"    - Value of two_matrix[{d}, {i}, {j}, 0]: {value_from_two_matrix:.6f}")
    
    # Use numpy.isclose for floating point comparison to avoid precision issues
    if np.isclose(value_from_one_matrix, value_from_two_matrix):
        print("    - Result: ✅ Consistent")
    else:
        print("    - Result: ❌ Inconsistent!")
        all_passed = False
    print() # Add empty line for readability

print("-" * 70)
if all_passed:
    print("Conclusion: All test cases passed, verifying that values from one_matrix are correctly used as the first part of two_matrix.")
else:
    print("Conclusion: Inconsistencies found.")

--- 验证关系: one_matrix[d, i] == two_matrix[d, i, j, 0] ---
----------------------------------------------------------------------
[*] 正在检查: 司机(d)=0, 第一个订单(i)=5, 第二个订单(j)=10
    - one_matrix[0, 5] 的值: 0.000000
    - two_matrix[0, 5, 10, 0] 的值: 0.000000
    - 结果: ✅ 一致

[*] 正在检查: 司机(d)=7, 第一个订单(i)=12, 第二个订单(j)=3
    - one_matrix[7, 12] 的值: 0.000000
    - two_matrix[7, 12, 3, 0] 的值: 0.000000
    - 结果: ✅ 一致

[*] 正在检查: 司机(d)=14, 第一个订单(i)=29, 第二个订单(j)=0
    - one_matrix[14, 29] 的值: 20.392405
    - two_matrix[14, 29, 0, 0] 的值: 20.392405
    - 结果: ✅ 一致

----------------------------------------------------------------------
结论: 所有测试用例均通过，验证了 one_matrix 的值被正确用作 two_matrix 的第一部分。


In [ ]:
print("\n--- Verify relationship: two_matrix[d, i, j, :] == three_matrix[d, i, j, k, 0:2] ---")
print("-" * 80)

# num_drivers and num_orders already obtained, can be used directly here

# Define some random test cases (driver index, order 1, order 2, order 3)
test_cases_3d = [
    (1, 2, 3, 4),
    (8, 15, 20, 5),
    (13, 28, 1, 29) # Another set of boundary cases
]

all_passed_3d = True

for d, i, j, k in test_cases_3d:
    # Ensure test case indices do not exceed matrix boundaries
    if d >= num_drivers or i >= num_orders or j >= num_orders or k >= num_orders:
        print(f"[*] Skipping test case ({d}, {i}, {j}, {k}) because indices are out of range.")
        continue

    print(f"[*] Checking: Driver(d)={d}, Path(i->j->k)={i}->{j}->{k}")

    # Get cost vector for path i -> j from two_matrix [first leg delay, total delay]
    vector_from_two_matrix = two_matrix[d, i, j, :]
    
    # Get first two elements of cost vector for path i -> j -> k from three_matrix
    vector_from_three_matrix = three_matrix[d, i, j, k, 0:2]
    
    # Use np.array2string to format output for easy comparison
    print(f"    - Value of two_matrix[{d}, {i}, {j}, :]:\t\t{np.array2string(vector_from_two_matrix, precision=6)}")
    print(f"    - Value of three_matrix[{d}, {i}, {j}, {k}, 0:2]:\t{np.array2string(vector_from_three_matrix, precision=6)}")
    
    # Use numpy.allclose to compare if two vectors are completely equal within tolerance
    if np.allclose(vector_from_two_matrix, vector_from_three_matrix):
        print("    - Result: ✅ Consistent")
    else:
        print("    - Result: ❌ Inconsistent!")
        all_passed_3d = False
    print()

print("-" * 80)
if all_passed_3d:
    print("Conclusion: All test cases passed, verifying that values from two_matrix are correctly used as the base part of three_matrix.")
else:
    print("Conclusion: Inconsistencies found.")


--- 验证关系: two_matrix[d, i, j, :] == three_matrix[d, i, j, k, 0:2] ---
--------------------------------------------------------------------------------
[*] 正在检查: 司机(d)=1, 路径(i->j->k)=2->3->4
    - two_matrix[1, 2, 3, :] 的值:		[ 0.       23.180733]
    - three_matrix[1, 2, 3, 4, 0:2] 的值:	[ 0.       23.180733]
    - 结果: ✅ 一致

[*] 正在检查: 司机(d)=8, 路径(i->j->k)=15->20->5
    - two_matrix[8, 15, 20, :] 的值:		[ 9.15875 35.58653]
    - three_matrix[8, 15, 20, 5, 0:2] 的值:	[ 9.15875 35.58653]
    - 结果: ✅ 一致

[*] 正在检查: 司机(d)=13, 路径(i->j->k)=28->1->29
    - two_matrix[13, 28, 1, :] 的值:		[0.      6.57865]
    - three_matrix[13, 28, 1, 29, 0:2] 的值:	[0.      6.57865]
    - 结果: ✅ 一致

--------------------------------------------------------------------------------
结论: 所有测试用例均通过，验证了 two_matrix 的值被正确用作 three_matrix 的基础部分。


In [ ]:
import pickle

# --- 1. Configuration Area (!!IMPORTANT!!) ---
# Please ensure parameters here are *exactly consistent* with those used when generating cost_matrices_30.npz file
ORDER_NUMBER = 30
# According to main.py, max_driver_number is the largest driver number in scenarios
MAX_DRIVER_NUMBER = 15 
# According to main.py, the seed used for the first scenario is 202
SEED = 202

# Copy seed generation logic from main.py to ensure consistency
ORDER_SEED = SEED + ORDER_NUMBER * 100000 + MAX_DRIVER_NUMBER * 1000
DRIVER_SEED = SEED + ORDER_NUMBER * 100000 + MAX_DRIVER_NUMBER * 1000

# Relative path from notebook to Resources folder
RESOURCES_PATH = "../Resources/test_ID/"


# --- 2. Directly load list file (without using DataLoader) ---
print("Directly loading list file to create ID mapping...")
try:
    # Build order_list filename and load
    order_list_path = f"{RESOURCES_PATH}order_list_{ORDER_SEED}_{ORDER_NUMBER}.pkl"
    with open(order_list_path, "rb") as f:
        order_list = pickle.load(f)

    # Build driver_list filename and load
    driver_list_path = f"{RESOURCES_PATH}driver_list_{DRIVER_SEED}_{ORDER_NUMBER}_{MAX_DRIVER_NUMBER}.pkl"
    with open(driver_list_path, "rb") as f:
        driver_list = pickle.load(f)

    # Create sorted ID list, this is the basis for indexing (and ensure they are strings)
    sorted_driver_ids = sorted([str(d) for d in driver_list])
    sorted_order_ids = sorted([str(o) for o in order_list])
    print("ID mapping loaded successfully!\n")

    # --- 3. Define indices you want to query ---
    DRIVER_INDICES_TO_FIND = [2]
    ORDER_INDICES_TO_FIND = [22,29]

    # --- 4. Execute and print query results ---
    print("--- ID Query Results ---")
    print("Driver:")
    for idx in DRIVER_INDICES_TO_FIND:
        if idx < len(sorted_driver_ids):
            print(f"  Index {idx} -> ID '{sorted_driver_ids[idx]}'")
        else:
            print(f"  Index {idx} -> (Out of range)")
    
    print("\nOrder:")
    for idx in ORDER_INDICES_TO_FIND:
        if idx < len(sorted_order_ids):
            print(f"  Index {idx:2d} -> ID '{sorted_order_ids[idx]}'")
        else:
            print(f"  Index {idx} -> (Out of range)")
    print("--------------------")

except FileNotFoundError as e:
    print(f"\nError: File not found - {e}")
    print("Please carefully check the 'Configuration Area' above to ensure all parameters and paths are correct.")

正在直接加载 list 文件以创建ID映射...
ID映射加载成功！

--- ID 查询结果 ---
司机 (Driver):
  索引 2 -> ID '123701'

订单 (Order):
  索引 22 -> ID '275658788056750.0'
  索引 29 -> ID '275659866257629.0'
--------------------
